<a href="https://colab.research.google.com/github/rar8393/NLG_tests/blob/main/NLG_opt_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

#model = AutoModelForCausalLM.from_pretrained("facebook/opt-30b", torch_dtype=torch.float16).cuda()

# the fast tokenizer currently does not work correctly
#tokenizer = AutoTokenizer.from_pretrained("facebook/opt-30b", use_fast=False)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [7]:
from transformers import pipeline, set_seed

set_seed(32)
generator = pipeline('text-generation', max_new_tokens=250, model="facebook/opt-350m", do_sample=True)
generator("Hello, I'm am conscious and")

[{'generated_text': "Hello, I'm am conscious and curious who you are, and if you have anything to say for now. I'll be trying to make some money this year, and am interested in learning more about finance and investing. I'm interested in the idea of building it into an investment, and maybe doing my first home equity loan.  So, any pointers on how to go about getting started? Maybe I can contact you?"}]

In [9]:
event_txt = 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! '
sick_txt = 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will '
generator(event_txt + sick_txt)

[{'generated_text': 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will ive a great community gathering!\n\nAmen! I am in attendance at the webinars, it is a great thing!\nAmen. There is too much at stake for just an "official" event, so I am very glad you have organized it.\n\nThe flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will ive a great community gathering!\n\nI also attended the webinar. It was a great time. We will now take suggestions for next year and be working on this soon. We started an article that describes how we are devel

In [10]:
event_txt = 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers.  '
sick_txt = 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will '
generator(event_txt + sick_txt)

[{'generated_text': 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers.  The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will  be the first time we will be holding a webinar.  For further information can PM or email.\nWould love to help, please send us a PM'}]